In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("from_hdfs_to_clickhouse")
    .master("spark://spark-master:7077")
    .config(
        "spark.jars.packages",
        ",".join([
            # clickhouse drivers
            "com.clickhouse.spark:clickhouse-spark-runtime-3.4_2.12:0.8.0",
            "com.clickhouse:clickhouse-client:0.7.0",
            "com.clickhouse:clickhouse-http-client:0.7.0",
            "org.apache.httpcomponents.client5:httpclient5:5.2.1",
        ])
    )
    # clickhouse catalog conf
    .config("spark.sql.catalog.clickhouse", "com.clickhouse.spark.ClickHouseCatalog")
    .config("spark.sql.catalog.clickhouse.host", "clickhouse-server")
    .config("spark.sql.catalog.clickhouse.protocol", "http")
    .config("spark.sql.catalog.clickhouse.http_port", "8123")
    .config("spark.sql.catalog.clickhouse.user", "default")
    .config("spark.sql.catalog.clickhouse.password", "1234qwe")
    .config("spark.sql.catalog.clickhouse.database", "default")
    .config("spark.clickhouse.write.format", "json")
    .getOrCreate()
)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
com.clickhouse.spark#clickhouse-spark-runtime-3.4_2.12 added as a dependency
com.clickhouse#clickhouse-client added as a dependency
com.clickhouse#clickhouse-http-client added as a dependency
org.apache.httpcomponents.client5#httpclient5 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-acccd4cf-af47-4b2b-ab77-d0a38fa37cf1;1.0
	confs: [default]
	found com.clickhouse.spark#clickhouse-spark-runtime-3.4_2.12;0.8.0 in central
	found com.clickhouse#clickhouse-client;0.7.0 in central
	found com.clickhouse#clickhouse-data;0.7.0 in central
	found com.clickhouse#clickhouse-http-client;0.7.0 in central
	found org.apache.httpcomponents.client5#httpclient5;5.2.1 in central
	found org.apache.httpcomponents.core5#httpcore5;5.2 in central
	found org.apache.httpcomponents.core5#httpcore5-h2;5.2 in central
	found org.slf4j#slf4j-api;1.7.36 in central
down

In [2]:
df = spark.read.csv("hdfs://namenode:9000/data/openbeer/breweries/breweries.csv", header=True, inferSchema=True)
df.show()

+---+--------------------+-------------+-----+---+
|_c0|                name|         city|state| id|
+---+--------------------+-------------+-----+---+
|  0|  NorthGate Brewing |  Minneapolis|   MN|  0|
|  1|Against the Grain...|   Louisville|   KY|  1|
|  2|Jack's Abby Craft...|   Framingham|   MA|  2|
|  3|Mike Hess Brewing...|    San Diego|   CA|  3|
|  4|Fort Point Beer C...|San Francisco|   CA|  4|
|  5|COAST Brewing Com...|   Charleston|   SC|  5|
|  6|Great Divide Brew...|       Denver|   CO|  6|
|  7|    Tapistry Brewing|     Bridgman|   MI|  7|
|  8|    Big Lake Brewing|      Holland|   MI|  8|
|  9|The Mitten Brewin...| Grand Rapids|   MI|  9|
| 10|      Brewery Vivant| Grand Rapids|   MI| 10|
| 11|    Petoskey Brewing|     Petoskey|   MI| 11|
| 12|  Blackrocks Brewery|    Marquette|   MI| 12|
| 13|Perrin Brewing Co...|Comstock Park|   MI| 13|
| 14|Witch's Hat Brewi...|   South Lyon|   MI| 14|
| 15|Founders Brewing ...| Grand Rapids|   MI| 15|
| 16|   Flat 12 Bierwerks| Indi

In [3]:
spark.sql("""
CREATE TABLE IF NOT EXISTS clickhouse.default.test_table (
    id INT NOT NULL,
    name STRING,
    city STRING
)
USING clickhouse
TBLPROPERTIES (
    'engine'='MergeTree()',
    'order_by'='id'
)
""")

DataFrame[]

In [4]:
df.writeTo("clickhouse.default.test_table").append()

In [5]:
df = spark.sql("select * from clickhouse.default.test_table")
df.show()

+---+--------------------+-------------+
| id|                name|         city|
+---+--------------------+-------------+
|  0|  NorthGate Brewing |  Minneapolis|
|  1|Against the Grain...|   Louisville|
|  2|Jack's Abby Craft...|   Framingham|
|  3|Mike Hess Brewing...|    San Diego|
|  4|Fort Point Beer C...|San Francisco|
|  5|COAST Brewing Com...|   Charleston|
|  6|Great Divide Brew...|       Denver|
|  7|    Tapistry Brewing|     Bridgman|
|  8|    Big Lake Brewing|      Holland|
|  9|The Mitten Brewin...| Grand Rapids|
| 10|      Brewery Vivant| Grand Rapids|
| 11|    Petoskey Brewing|     Petoskey|
| 12|  Blackrocks Brewery|    Marquette|
| 13|Perrin Brewing Co...|Comstock Park|
| 14|Witch's Hat Brewi...|   South Lyon|
| 15|Founders Brewing ...| Grand Rapids|
| 16|   Flat 12 Bierwerks| Indianapolis|
| 17|Tin Man Brewing C...|   Evansville|
| 18|Black Acre Brewin...| Indianapolis|
| 19|   Brew Link Brewing|   Plainfield|
+---+--------------------+-------------+
only showing top

In [6]:
spark.stop()